# Numerical Taxonomy of Urban Form in El-Paso and Ciudad-Juares

This notebook serves as a simplify template for morphometric assessment and generation of a taxonomy.

## Reproducible Python code to generate taxonomy

Complete morphometrics assessment from input data to taxonomy.

Input data:
 - building footprints
 - street network
 
This notebook is running the analysis on the sample of the data used in El-Paso and Ciudad-Juares case study. You can replace the sample with your own data, assuming that they are cleaned to a required standard. 

The sample is saved in `../data/data.gpkg` with two layers named `buildings` and `streets`. `buildings` are Polygons, whilst `streets` are LineStrings. They don't have any additonal attribuets.

All data generated throughout the method are saved to files (unless commented out).

This work demand at least 128GB RAM and 12GB of storage.

First we import all required libraries.

In [ ]:
import warnings

import geopandas as gpd
import libpysal
import mapclassify
import matplotlib.pyplot as plt
import momepy as mm
import numpy as np
import pandas as pd
import scipy as sp
import seaborn as sns

from tqdm.auto import tqdm
from sklearn import preprocessing
from sklearn.mixture import GaussianMixture
from scipy.cluster import hierarchy

# we are using bleeding edge software that emits some warnings irrelevant for the current runtime
warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
warnings.filterwarnings('ignore', message='.*overflow encountered*')
warnings.filterwarnings('ignore', message='.*index_parts defaults to True')
warnings.filterwarnings('ignore', message='.*`op` parameter is deprecated*')

### Check the input data

We load buildings and create unique ID and change crs if necessary.

In [ ]:
path = "../data/data.gpkg"
layer = "buildings"
buildings = gpd.read_file(path, layer=layer)
buildings = buildings.to_crs(crs=3857)

Let's create a persistent unique identifier for each building.

In [ ]:
buildings["uID"] = range(len(buildings))

### Generate additional morphometric elements

Before we can start morhometrics we have to generate additional elements - tessellation and tessellation based blocks.

#### Morphological tessellation

Check input for tessellation. If the input data is clean, the check will result in zeros. The data does not have to be 100% clean (all 0). For example `Split features` may not cause any issue. The function `mm.preprocess` allows you to eliminate some problems with geometry.

In [ ]:
check = mm.CheckTessellationInput(buildings)
buildings = mm.preprocess(buildings.reset_index(drop=True), size=10, compactness=0.2, loops=2, islands=True)

Generate tessellation limited to 100 m buffer. Beware, it is memory demanding.

**Note:** Code is adaptated for enclosed tessellation. You can do other method which requires more memory (more than 180GB for the study). In that case, you may consider using an [morphological tessellation](https://docs.momepy.org/en/stable/user_guide/elements/tessellation.html) method instead. However, that would require minor adaptation of the code below as well.

First we need to import streets and change crs if necessary.

In [ ]:
streets = gpd.read_file(path, layer='streets')
streets = streets.to_crs(crs=3857)

Creating enclosures.

In [ ]:
limit = mm.buffered_limit(buildings, 100)
enclosures = mm.enclosures(streets, limit=gpd.GeoSeries([limit]))

Creating tessellation.

In [ ]:
tessellation = mm.Tessellation(buildings, unique_id='uID', enclosures=enclosures)
tessellation = tessellation.tessellation

We save tessellation to file. Note that this file is not part of the repository but can be fully created using the input sample and this notebook.

In [ ]:
tessellation.to_file("../data/geometry.gpkg", layer="tessellation")

### Link streets

We need to understand which building belongs to which street segment. We link IDs together based on proximity.

In [ ]:
streets["nID"] = range(len(streets))
buildings['nID'] = mm.get_network_id(buildings, streets, 'nID', min_size=300, verbose=False)

### Repair data

Drop null nIDs in buildings.

In [ ]:
buildings = buildings.dropna(subset=['nID'])

Drop unnecessary elements in `buildings` and `tessellation`. 

In [ ]:
tessellation = tessellation.explode().drop_duplicates(subset=['uID'])

buildings = buildings.merge(tessellation[['uID']], on='uID', how='inner')
tessellation = tessellation.merge(buildings[['uID', 'nID']], on='uID', how='inner')

buildings = buildings.drop_duplicates(subset='uID')
tessellation = tessellation.drop_duplicates(subset='uID')

Check if lengths of both frames is equal.

In [ ]:
print(len(tessellation))
print(len(buildings))

Finally, we save elements to a file.

In [ ]:
path = '../data/geometry.gpkg'
tessellation.to_file(path, layer='tessellation', driver='GPKG')
buildings.to_file(path, layer='buildings', driver='GPKG')
streets.to_file(path, layer='streets', driver='GPKG')

### Creating blocks

To create blocks within the full limit, it is always safer to extend street network to the edge of the limit.

In [ ]:
extended = mm.extend_lines(streets, tolerance=120, target=gpd.GeoSeries([limit.boundary]), barrier=buildings)

If you are confident in your data, you can simply create blocks by using `mm.Blocks` class which is commented below. But I will recomend doing it manually.

In [ ]:
#blocks = mm.Blocks(tessellation, edges=extended, buildings=buildings, id_name='bID', unique_id='uID')
#blocks_df = blocks.blocks
#buildings['bID'] = blocks.buildings_id.values
#tessellation['bID'] = blocks.tessellation_id.values 

#blocks = blocks_df

Manual method.

In [ ]:
cut = gpd.overlay(tessellation, gpd.GeoDataFrame(geometry=extended.buffer(0.001)), how="difference")

In [ ]:
cut = cut.explode(ignore_index=True)
weights = libpysal.weights.Queen.from_dataframe(cut, silence_warnings=True)

cut["component"] = weights.component_labels
buildings_c = buildings.copy()
buildings_c.geometry = buildings_c.representative_point()  # make points

centroids_temp_id = gpd.sjoin(
        buildings_c,
        cut[[cut.geometry.name, "component"]],
        how="left",
        predicate="within",
)

In [ ]:
cells_copy = tessellation[['uID', tessellation.geometry.name]].merge(centroids_temp_id[['uID', "component"]], on='uID', how="left")
blocks = cells_copy.dissolve(by="component").explode(ignore_index=True)

In [ ]:
blocks['bID'] = range(len(blocks))
blocks = blocks[['bID', blocks.geometry.name]]

In [ ]:
centroids_w_bl_id2 = gpd.sjoin(buildings_c, blocks, how="left", predicate="within")
buildings_id = centroids_w_bl_id2['bID']

In [ ]:
cells_m = tessellation[['uID']].merge(centroids_w_bl_id2[['uID', 'bID']], on='uID', how="left")
cells_m = cells_m.drop_duplicates(subset='uID')

In [ ]:
buildings = buildings.merge(cells_m[['uID', 'bID']], on='uID', how='left')
tessellation = tessellation.merge(cells_m[['uID', 'bID']], on='uID', how='left')

Save data.

In [ ]:
tessellation.to_file(path, layer='tessellation', driver='GPKG')
buildings.to_file(path, layer='buildings', driver='GPKG')
streets.to_file(path, layer='streets', driver='GPKG')
blocks.to_file(path, layer='blocks', driver='GPKG')

### Creating weights

In [ ]:
queen_1 = libpysal.weights.contiguity.Queen.from_dataframe(tessellation, ids="uID", silence_warnings=True)
buildings_q1 = libpysal.weights.contiguity.Queen.from_dataframe(buildings, silence_warnings=True)
blo_q1 = libpysal.weights.contiguity.Queen.from_dataframe(blocks, ids="bID", silence_warnings=True)
queen_3 = mm.sw_high(k=3, weights=queen_1)
str_q1 = libpysal.weights.contiguity.Queen.from_dataframe(streets, silence_warnings=True)

In [ ]:
path = '../data/'

fo = libpysal.io.open(path+'queen_1.gal', 'w')
fo.write(queen_1)
fo.close()

fo = libpysal.io.open(path+'buildings_q1.gal', 'w')
fo.write(buildings_q1)
fo.close()

fo = libpysal.io.open(path+'blo_q1.gal', 'w')
fo.write(blo_q1)
fo.close()

fo = libpysal.io.open(path+'queen_3.gal', 'w')
fo.write(queen_3)
fo.close()

fo = libpysal.io.open(path+'str_q1.gal', 'w')
fo.write(str_q1)
fo.close()

## Measure primary characters

This part measures 55 primary morphometric characters.

It does save intermediate parquet files as a backup.

In [ ]:
buildings['sdbAre'] = mm.Area(buildings).series
buildings['sdbPer'] = mm.Perimeter(buildings).series
buildings['sdbCoA'] = mm.CourtyardArea(buildings, 'sdbAre').series

buildings['ssbCCo'] = mm.CircularCompactness(buildings, 'sdbAre').series
buildings['ssbCor'] = mm.Corners(buildings, verbose=False).series
buildings['ssbSqu'] = mm.Squareness(buildings, verbose=False).series
buildings['ssbERI'] = mm.EquivalentRectangularIndex(buildings, 'sdbAre', 'sdbPer').series
buildings['ssbElo'] = mm.Elongation(buildings).series

In [ ]:
cencon = mm.CentroidCorners(buildings, verbose=False)
buildings['ssbCCM'] = cencon.mean
buildings['ssbCCD'] = cencon.std

In [ ]:
buildings['stbOri'] = mm.Orientation(buildings, verbose=False).series
tessellation['stcOri'] = mm.Orientation(tessellation, verbose=False).series

buildings['stbCeA'] = mm.CellAlignment(buildings, tessellation, 'stbOri', 'stcOri', 'uID', 'uID').series

In [ ]:
tessellation['sdcLAL'] = mm.LongestAxisLength(tessellation).series
tessellation['sdcAre'] = mm.Area(tessellation).series
tessellation['sscCCo'] = mm.CircularCompactness(tessellation, 'sdcAre').series
tessellation['sscERI'] = mm.EquivalentRectangularIndex(tessellation, 'sdcAre').series

In [ ]:
buildings["libNCo"] = mm.Courtyards(buildings, spatial_weights=buildings_q1, verbose=False).series
buildings["ldbPWL"] = mm.PerimeterWall(buildings, buildings_q1, verbose=False).series

blocks["ldkAre"] = mm.Area(blocks).series
blocks["ldkPer"] = mm.Perimeter(blocks).series
blocks["lskCCo"] = mm.CircularCompactness(blocks, "ldkAre").series
blocks["lskERI"] = mm.EquivalentRectangularIndex(blocks, "ldkAre", "ldkPer").series
blocks["lskCWA"] = mm.CompactnessWeightedAxis(blocks, "ldkAre", "ldkPer").series
blocks["ltkOri"] = mm.Orientation(blocks, verbose=False).series

blocks["ltkWNB"] = mm.Neighbors(blocks, blo_q1, "bID", weighted=True, verbose=False).series
blocks["likWBB"] = mm.Count(blocks, buildings, "bID", "bID", weighted=True).series

Save data to parquets as a checkpoint backup.

In [ ]:
tessellation.drop(columns='geometry').to_parquet('../data/tess_data.parquet')
buildings.drop(columns='geometry').to_parquet('../data/buildings_data.parquet')
blocks.drop(columns='geometry').to_parquet('../data/blocks_data.parquet')

In [ ]:
buildings['ltbIBD'] = mm.MeanInterbuildingDistance(buildings, queen_1, 'uID', queen_3, verbose=False).series
buildings['ltcBuA'] = mm.BuildingAdjacency(buildings, queen_3, 'uID', buildings_q1, verbose=False).series

In [ ]:
tessellation['sicCAR'] = mm.AreaRatio(tessellation, buildings, 'sdcAre', 'sdbAre', 'uID').series

Save data to parquets and spatial weights matrices to gal files.

In [ ]:
tessellation.drop(columns='geometry').to_parquet('../data/tess_data.parquet')
buildings.drop(columns='geometry').to_parquet('../data/buildings_data.parquet')
blocks.drop(columns='geometry').to_parquet('../data/blocks_data.parquet')

In [ ]:
streets["sdsLen"] = mm.Perimeter(streets).series

In [ ]:
streets["sssLin"] = mm.Linearity(streets).series
streets["sdsAre"] = mm.Reached(streets, tessellation, "nID", "nID", mode="sum", values="sdcAre").series
streets["sisBpM"] = mm.Count(streets, buildings, "nID", "nID", weighted=True).series

In [ ]:
tessellation.drop(columns='geometry').to_parquet('../data/tess_data.parquet')
buildings.drop(columns='geometry').to_parquet('../data/buildings_data.parquet')
streets.drop(columns='geometry').to_parquet('../data/streets_data.parquet')
blocks.drop(columns='geometry').to_parquet('../data/blocks_data.parquet')

In [ ]:
streets["misRea"] = mm.Reached(streets, tessellation, "nID", "nID", spatial_weights=str_q1, mode="count", verbose=False).series
streets["mdsAre"] = mm.Reached(streets, tessellation, "nID", "nID", spatial_weights=str_q1, mode="sum", verbose=False).series

In [ ]:
graph = mm.gdf_to_nx(streets.explode())
graph = mm.node_degree(graph)
graph = mm.subgraph(
    graph,
    radius=5,
    meshedness=True,
    cds_length=False,
    mode="sum",
    degree="degree",
    length="mm_len",
    mean_node_degree=False,
    proportion={0: True, 3: True, 4: True},
    cyclomatic=False,
    edge_node_ratio=False,
    gamma=False,
    local_closeness=True,
    closeness_weight="mm_len",
    verbose=False
)
graph = mm.cds_length(graph, radius=3, name="ldsCDL", verbose=False)
graph = mm.clustering(graph, name="xcnSCl")
graph = mm.mean_node_dist(graph, name="mtdMDi", verbose=False)

nodes, edges, sw = mm.nx_to_gdf(graph, spatial_weights=True)

In [ ]:
nodes.to_file(path+'geometry.gpkg', layer="nodes", driver="GPKG")
edges.to_file(path+'geometry.gpkg', layer="edges", driver="GPKG")

fo = libpysal.io.open("../data/nodes.gal", "w")
fo.write(sw)
fo.close()

In [ ]:
edges_w3 = mm.sw_high(k=3, gdf=edges)
edges["ldsMSL"] = mm.SegmentsLength(edges, spatial_weights=edges_w3, mean=True, verbose=False).series

edges["ldsRea"] = mm.Reached(edges, tessellation, "nID", "nID", spatial_weights=edges_w3, verbose=False).series
edges["ldsRea"] = mm.Reached(edges, tessellation, "nID", "nID", spatial_weights=edges_w3, mode="sum", values="sdcAre", verbose=False).series

nodes_w5 = mm.sw_high(k=5, weights=sw)
nodes["lddNDe"] = mm.NodeDensity(nodes, edges, nodes_w5, verbose=False).series
nodes["linWID"] = mm.NodeDensity(nodes, edges, nodes_w5, weighted=True, node_degree="degree", verbose=False).series

buildings["nodeID"] = mm.get_node_id(buildings, nodes, edges.drop_duplicates(subset='nID'), "nodeID", "nID")
tessellation = tessellation.merge(buildings[["uID", "nodeID"]], on="uID", how="left")

nodes_w3 = mm.sw_high(k=3, weights=sw)

nodes["lddRea"] = mm.Reached(nodes, tessellation, "nodeID", "nodeID", nodes_w3, verbose=False).series
nodes["lddARe"] = mm.Reached(nodes, tessellation, "nodeID", "nodeID", nodes_w3, mode="sum", values="sdcAre", verbose=False).series

nodes["sddAre"] = mm.Reached(nodes, tessellation, "nodeID", "nodeID", mode="sum", values="sdcAre", verbose=False).series
nodes["midRea"] = mm.Reached(nodes, tessellation, "nodeID", "nodeID", spatial_weights=sw, verbose=False).series
nodes["midAre"] = mm.Reached(nodes, tessellation, "nodeID", "nodeID", spatial_weights=sw, mode="sum", values="sdcAre", verbose=False).series

nodes.rename(
    columns={
        "degree": "mtdDeg",
        "meshedness": "lcdMes",
        "local_closeness": "lcnClo",
        "proportion_3": "linP3W",
        "proportion_4": "linP4W",
        "proportion_0": "linPDE",
    }, inplace=True
)

In [ ]:
tessellation.drop(columns='geometry').to_parquet('../data/tess_data.parquet')
buildings.drop(columns='geometry').to_parquet('../data/buildings_data.parquet')
blocks.drop(columns='geometry').to_parquet('../data/blocks_data.parquet')
nodes.drop(columns='geometry').to_parquet('../data/nodes_data.parquet')
edges.drop(columns='geometry').to_parquet('../data/edges_data.parquet')

In [ ]:
merged = tessellation.merge(buildings.drop(columns=['nID', 'bID', 'nodeID', 'geometry']), on='uID')
merged = merged.merge(blocks.drop(columns='geometry'), on='bID', how='left')
merged = merged.merge(edges.drop(columns='geometry'), on='nID', how='left')
merged = merged.merge(nodes.drop(columns='geometry'), on='nodeID', how='left')

Clean columns to keep only measured data.

In [ ]:
primary = merged.drop(columns=['nID', 'bID', 'eID', 'nodeID', 'mm_len', 'cdsbool', 'node_start', 'node_end', 'geometry'])

In [ ]:
primary.to_parquet('../data/primary.parquet')

## Measure contextual - spatially lagged characters

This part measures contextual characters.

In [ ]:
gdf = primary.set_index('uID')
spatial_weights = queen_3
unique_id = 'uID'

For importing `spatial_weights` from file use the code below.

In [ ]:
#spatial_weights = libpysal.io.open('../data/queen_3.gal', 'r').read()
#spatial_weights.neighbors = {int(float(k)): [int(float(i)) for i in v] for k, v in spatial_weights.neighbors.items()}

Resolve potential missingness cause by invalid input data. That was not case in the presented case studies but may be case in subsequent research.

In [ ]:
gdf = gdf.replace(np.inf, np.nan).fillna(0)

In [ ]:
gdf['lcdMes'] = gdf.apply(lambda row: row.lcdMes if row.lcdMes >= 0 else 0, axis=1)  # normally does not happen, but to be sure

In [ ]:
indexes = list(gdf.index.values)
indexes_set = frozenset(indexes)
all_indexes = list(range(int(gdf.index.max())))
new_indexes = [i for i in all_indexes if i not in indexes_set]
gdf = pd.concat([gdf, pd.DataFrame(np.nan, new_indexes, chars)])
gdf = gdf.sort_index()
ndf = gdf.to_numpy()

Define Theil function.

In [ ]:
def _theil(y):
    y = np.array(y)
    n = len(y)
    plus = y + np.finfo('float').tiny * (y == 0)  # can't have 0 values
    yt = plus.sum(axis=0)
    s = plus / (yt * 1.0)
    lns = np.log(n * s)
    slns = s * lns
    t = sum(slns)
    return t

Loop over DataFrame and measure IQM, IQR and IDT.

In [ ]:
means = []
ranges = []
theils = []

In [ ]:
def limit_range(subset, rng_min, rng_max):
    lower = np.nanpercentile(subset, rng_min)
    higher = np.nanpercentile(subset, rng_max)
    new = np.where((lower <= subset)&(subset <= higher), subset, np.nan)
    return new

In [ ]:
for index in tqdm(indexes, total=len(indexes)):
    
    subset = ndf[np.array([int(x) for x in ([index] + spatial_weights.neighbors[index])]), :]
    
    means.append(np.nanmean(limit_range(subset, 25, 70), axis=0))
    ranges.append(np.nanmax(limit_range(subset, 25, 70), axis=0) - np.nanmin(limit_range(subset, 25, 70), axis=0)) 
    theils.append(_theil(limit_range(subset, 10, 90)))

Get final contextual data.

In [ ]:
columns = gdf.columns

means = pd.DataFrame(means[0:], columns=columns+'_IQM', index=indexes)
ranges = pd.DataFrame(ranges[0:], columns=columns+'_IQR', index=indexes)
theils = pd.DataFrame(theils[0:], columns=columns+'_IDT', index=indexes)

In [ ]:
gdf = gdf.drop(new_indexes)

In [ ]:
contextual = means_df.join(ranges, how='inner').join(theils, how='inner')

In [ ]:
contextual = contextual.dropna(axis=1, how='all')
contextual = contextual.dropna(axis=0, how='all')
contextual = contextual[~contextual.index.duplicated()].copy()

In [ ]:
contextual.shape

In [ ]:
contextual.to_parquet('../data/contextual.parquet')

## Clustering

**Note**: This part of work you can handle in [Kaggle](https://www.kaggle.com) which is designed for machine learning purpose.

We use contextual characters to do GMM clustering.

In [ ]:
data = contextual.copy()

First we standardize data.

In [ ]:
x = data.values
scaler = preprocessing.StandardScaler()
cols = list(data.columns)
data[cols] = scaler.fit_transform(data[cols])

Measure BIC to estimate optimal number of clusters.

In [ ]:
bic = pd.DataFrame(columns=['n', 'bic', 'run'])
ix = 0

n_components_range = range(2, 8) # specify range you want to assess
gmmruns = 1  # specify how many times should each option be tried (more better, but takes a long time)

In [ ]:
data = data.fillna(0)
for n_components in n_components_range:
    for i in range(gmmruns):
        gmm = GaussianMixture(n_components=n_components, covariance_type="full", max_iter=200, n_init=1, verbose=1)
        fitted = gmm.fit(data)
        bicnum = gmm.bic(data)
        bic.loc[ix] = [n_components, bicnum, i]
        ix += 1

        print(n_components, i, "BIC:", bicnum)

In [ ]:
bic.to_csv('../data/complete_BIC.csv')

Based on the plot below, we estimate the optimal `n` either based on the elbow of the curve or as the minimum.

In [ ]:
fig, ax = plt.subplots(figsize=(16, 16))
sns.lineplot(ax=ax, x='n', y='bic', data=bic)
plt.savefig('../data/complete_BIC.pdf')

### Gaussian Mixture Model

In [ ]:
n = 5 # illustrative - always base the number on a reasonable estimation of the optimal number of components
n_init = 1  # more initialization, more stable clustering gets

gmm = GaussianMixture(n_components=n, covariance_type="full", max_iter=200, n_init=n_init, verbose=1)
fitted = gmm.fit(data)

In [ ]:
labels = gmm.predict(data)

In [ ]:
pd.Series(labels, index=data.index).to_csv('../data/cluster_labels.csv')

#### Hierachical clustering

Finally, we create hierarchical classification - taxonomy.

In [ ]:
group = data.groupby(labels).mean()
Z = hierarchy.linkage(group, 'ward')
plt.figure(figsize=(16, 9))
dn = hierarchy.dendrogram(Z, labels=group.index)

plt.savefig('tree.png')

## Results

In [ ]:
final = pd.DataFrame(labels, index=data.index)

In [ ]:
buildings = buildings.set_index('uID')
buildings = buildings.join(final)
buildings = buildings.rename(columns={0: 'Class'})
buildings = buildings.drop(columns=['nID', 'bID'])

In [ ]:
buildings.to_file('../data/buildings_labels.gpkg', layer='buildings', driver='GPKG')

In [ ]:
tessellation = tessellation.set_index('uID')
tessellation = tessellation.join(final)
tessellation = tessellation.rename(columns={0: 'Class'})
tessellation = tessellation.drop(columns=['nID', 'bID', 'eID'])

In [ ]:
tessellation.to_file('../data/tessellation_labels.gpkg', layer='tessellation', driver='GPKG')